In [1]:
import os
import requests
from datetime import datetime

#import geopandas as gpd
#import geojsonio

import pandas as pd
from pandas.io.json import json_normalize

import folium
from folium import plugins


## USGS GeoJSON format

schema = { type: "FeatureCollection", metadata: { generated: Long Integer, url: String, title: String, api: String, count: Integer, status: Integer }, bbox: [ minimum longitude, minimum latitude, minimum depth, maximum longitude, maximum latitude, maximum depth ], features: [ { type: "Feature", properties: { mag: Decimal, place: String, time: Long Integer, updated: Long Integer, tz: Integer, url: String, detail: String, felt:Integer, cdi: Decimal, mmi: Decimal, alert: String, status: String, tsunami: Integer, sig:Integer, net: String, code: String, ids: String, sources: String, types: String, nst: Integer, dmin: Decimal, rms: Decimal, gap: Decimal, magType: String, type: String }, geometry: { type: "Point", coordinates: [ longitude, latitude, depth ] }, id: String }, … ] }


## Getting data


In [2]:
data = requests.get("https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_day.geojson")
# data = requests.get("https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_week.geojson")


jsondata = data.json()
quakes = pd.json_normalize(jsondata['features'])



In [3]:
quakes.head(2)

,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,properties.felt,...,properties.types,properties.nst,properties.dmin,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates
0,Feature,us7000hfmx,4.50,"36 km SSE of Jurm, Afghanistan",1654621044454,1654628108058,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,1.0,...,",dyfi,origin,phase-data,",NaN,0.6540,0.47,72.0,mb,earthquake,"M 4.5 - 36 km SSE of Jurm, Afghanistan",Point,"[70.9979, 36.5639, 190.24]"
1,Feature,nc73741171,2.56,"43km S of Qualeys Camp, NV",1654619484490,1654628471040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,...,",focal-mechanism,nearby-cities,origin,phase-da...",44.0,0.1694,0.05,65.0,md,earthquake,"M 2.6 - 43km S of Qualeys Camp, NV",Point,"[-118.5233333, 37.925, 6.67]"


## Cleaning the data




Some of these fields are essientially useless in all cases. A good example is geometry.type which according to the schema definition is going to always be point.

A couple are just redundant. For example 'title' is just 'mag' plus 'place'

Some are probably useless but might be useful, particularly if we apply a ML model to it later. So, I'll keep them for now. But the prepending everything with 'properties' or 'geometry' -- that has to go.


In [6]:
quakes.columns = quakes.columns.str.replace('properties.', "", regex=False)
quakes.columns = quakes.columns.str.replace('geometry.', "", regex=False)
quakes.drop(['id', 'type', 'updated', 'tz', 'mmi', 'detail', 'felt','cdi', 'felt', 'types', 'nst', 'type', 'title'], 
        axis=1, inplace=True)

KeyError: "['id', 'type', 'updated', 'tz', 'mmi', 'detail', 'felt', 'cdi', 'felt', 'types', 'nst', 'type', 'title'] not found in axis"

In [7]:
quakes.head(2)

,mag,place,time,url,alert,status,tsunami,sig,net,code,ids,sources,dmin,rms,gap,magType,coordinates
0,4.50,"36 km SSE of Jurm, Afghanistan",1654621044454,https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,312,us,7000hfmx,",us7000hfmx,",",us,",0.6540,0.47,72.0,mb,"[70.9979, 36.5639, 190.24]"
1,2.56,"43km S of Qualeys Camp, NV",1654619484490,https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,101,nc,73741171,",nc73741171,nn00840479,us7000hfmu,",",nc,nn,us,",0.1694,0.05,65.0,md,"[-118.5233333, 37.925, 6.67]"


In [8]:


quakes['ids'] = quakes['ids'].str.strip(',')
quakes['sources'] = quakes['sources'].str.strip(',')



In [9]:
quakes['time'] = pd.to_datetime(quakes['time'], unit='ms')

In [10]:
quakes

,mag,place,time,url,alert,status,tsunami,sig,net,code,ids,sources,dmin,rms,gap,magType,coordinates
0,4.50,"36 km SSE of Jurm, Afghanistan",2022-06-07 16:57:24.454,https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,312,us,7000hfmx,us7000hfmx,us,0.65400,0.47,72.0,mb,"[70.9979, 36.5639, 190.24]"
1,2.56,"43km S of Qualeys Camp, NV",2022-06-07 16:31:24.490,https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,101,nc,73741171,"nc73741171,nn00840479,us7000hfmu","nc,nn,us",0.16940,0.05,65.0,md,"[-118.5233333, 37.925, 6.67]"
2,4.30,"282 km WNW of Attu Station, Alaska",2022-06-07 15:59:55.753,https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,284,us,7000hfmf,us7000hfmf,us,3.09600,0.40,133.0,mb,"[169.3483, 53.9412, 29.2]"
3,4.90,"59 km NE of Norsup, Vanuatu",2022-06-07 15:51:52.020,https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,370,us,7000hfmc,us7000hfmc,us,0.53100,0.67,74.0,mb,"[167.7222, -15.6271, 121.66]"
4,4.10,"96 km NW of Nikolski, Alaska",2022-06-07 14:55:20.010,https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,259,us,7000hfl3,"ak02279jkqhp,us7000hfl3","ak,us",0.83400,0.58,57.0,mb,"[-169.6895, 53.6475, 160.63]"
5,4.00,"Rat Islands, Aleutian Islands, Alaska",2022-06-07 14:11:45.935,https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,246,us,7000hfkt,"ak02279jbcx9,us7000hfkt","ak,us",0.28000,0.69,176.0,mb,"[178.2501, 51.7201, 112.95]"
6,3.20,"Rat Islands, Aleutian Islands, Alaska",2022-06-07 13:52:20.755,https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,158,us,7000hfkp,us7000hfkp,us,1.10700,0.51,236.0,ml,"[178.0739, 50.8941, 35]"
7,2.54,"22 km SSW of Esperanza, Puerto Rico",2022-06-07 13:45:55.620,https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,99,pr,71352773,pr71352773,pr,NaN,0.16,239.0,md,"[-65.5838333333333, 17.9233333333333, 34.96]"
8,5.60,"79 km ESE of Gorontalo, Indonesia",2022-06-07 13:24:13.673,https://earthquake.usgs.gov/earthquakes/eventp...,green,reviewed,0,483,us,7000hfjt,us7000hfjt,us,1.48600,0.85,22.0,mww,"[123.667, 0.1524, 129.11]"
9,4.80,"Pagan region, Northern Mariana Islands",2022-06-07 12:35:17.863,https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,354,us,7000hfjf,us7000hfjf,us,4.93400,0.56,82.0,mb,"[145.6996, 18.4869, 221.58]"


## parsing the data

Breakout the coords column


In [11]:
quakes['longitude'] = quakes.coordinates.str[0]
quakes['latitude'] = quakes.coordinates.str[1]
quakes['depth'] = quakes.coordinates.str[2]

quakes.drop(['coordinates'], axis=1, inplace=True)

In [12]:
quakes['datetime'] = pd.to_datetime(quakes['time']).dt.strftime("%Y-%m-%d %H:%M")
quakes.drop(['time'], axis=1, inplace=True)

In [13]:
quakes.round({'longitude': 5, 'latitude': 5, 'depth': 0})
quakes.head(2)


,mag,place,url,alert,status,tsunami,sig,net,code,ids,sources,dmin,rms,gap,magType,longitude,latitude,depth,datetime
0,4.50,"36 km SSE of Jurm, Afghanistan",https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,312,us,7000hfmx,us7000hfmx,us,0.6540,0.47,72.0,mb,70.997900,36.5639,190.24,2022-06-07 16:57
1,2.56,"43km S of Qualeys Camp, NV",https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,101,nc,73741171,"nc73741171,nn00840479,us7000hfmu","nc,nn,us",0.1694,0.05,65.0,md,-118.523333,37.9250,6.67,2022-06-07 16:31


In [14]:
m = folium.Map(
    location=[0, 0],
    tiles='cartodbpositron',
    zoom_start=1.8
)



Figure out how to get time to display


In [15]:
fmtr = "function(num) {return L.Util.formatNum(num, 3);};"
folium.plugins.MousePosition(separator=' / ', prefix="Lat/Long: ", lat_formatter=fmtr, lng_formatter=fmtr).add_to(m)


In [16]:
for i, row in quakes.iterrows():
    folium.CircleMarker((row.latitude, row.longitude), radius=row.mag*2.2, 
                        color='red', weight = 0, opacity =.4, fill=True, fill_color='orange', fill_opacity=.3, 
                        popup=[f"Time: {row.datetime},\n Mag: {row.mag},\n Depth: {row.depth} km"]).add_to(m)
    


In [17]:
boundaries = ('data/GeoJSON/PB2002_boundaries.json')
line_style = {'color': '#FF3333', 'weight': 1, 'opacity':row.mag*1.4}


folium.GeoJson(
    boundaries,
    name='major fault lines',
    style_function=lambda x:line_style,
    smooth_factor=4.0,
      
).add_to(m)
m


FileNotFoundError: [Errno 2] No such file or directory: 'data/GeoJSON/PB2002_boundaries.json'